## Import libraries and set random seed

In [17]:
from gurobipy import Model, GRB, tuplelist, quicksum
from gurobipy import read as gurobi_read_model
import json
import random
import math

random.seed(3)

## Set scheduler parameters

In [2]:
START_TIME = 0
SLICE_SIZE_MINUTES = 5
HORIZON_DAYS = 1

slice_size = SLICE_SIZE_MINUTES*60
horizon = HORIZON_DAYS*24*60*60
print(f"Slicesize: {slice_size}s")
print(f"Scheduling Horizon: {horizon}s")

Slicesize: 300s
Scheduling Horizon: 86400s


## Required Functions

### Possible Starts

In [24]:
class PossibleStart(object):
    def __init__(self, resource, slice_starts, internal_start):
        self.resource = resource
        self.first_slice_start = slice_starts[0]
        self.all_slice_starts = slice_starts
        self.internal_start = internal_start

    def __lt__(self, other):
        return self.first_slice_start < other.first_slice_start

    def __eq__(self, other):
        return self.first_slice_start == self.first_slice_start

    def __gt__(self, other):
        return self.first_slice_start > self.first_slice_start
        

def get_slices(intervals, resource, duration, slice_length):
    slices = []
    internal_starts = []
    for t in intervals: 
        start = int(math.floor(float(t["start"])/float(slice_length))*slice_length) # Start of the time_slice that this starts in
        internal_start = t["start"]                                                 # Actual start of this observation window
        end_time = internal_start + duration                                        # End of this observation window

        while (t["end"] - start) >= duration:
            tmp = range(start, internal_start+duration, slice_length)    # Generate a range of slices that will be occupied for this start and duration
            slices.append(tmp)
            internal_starts.append(internal_start)
            start += slice_length                                        # Moving on to the next potential start, so move the Start up to the next slice
            internal_start = start                                       # As only the first potential start in a window will be internal, make the 
                                                                         # next Internal Start an external start
    # return slices, internal_starts
    ps_list = []
    idx = 0
    for w in slices:
        ps_list.append(PossibleStart(resource, w, internal_starts[idx]))
        idx += 1

    return ps_list

#### Test

In [4]:
# TEST:
intervals = [{"start": 0, "end": 30}, {"start": 60, "end": 90}]
possible_starts = get_slices(intervals, 13, 3, slice_size)
print(possible_starts[1])

### Build Data Sctructures

Build the Yik and the aikt for generating the Optimizer Constraints from.

Requests must be a list of requests, which should have:
* free_windows_dict
* duration
* resID
* priority

#### New Code

In [12]:
def build_data_structures(requests, slice_length):
    yik = []
    aikt = {}

    for r in requests:
        yik_entries = []
        possible_starts = []
        for resource in r["free_windows_dict"].keys():
            possible_starts.extend(get_slices(r["free_windows_dict"][resource], resource, r["duration"], slice_length))
        possible_starts.sort()
    
        w_idx = 0
        for ps in possible_starts:
            yik_idx = len(yik)
            yik_entries.append(yik_idx)
            scheduled = 0 # Option to set to 1 for a Warm Start would go here
            yik.append([r["resID"], w_idx, r["priority"], ps.resource, scheduled])
            w_idx += 1
    
            for s in ps.all_slice_starts:
                slice_hash = f"resource_{ps.resource}_start_{repr(s)}_length_{repr(slice_length)}"
                if slice_hash not in aikt:
                    aikt[slice_hash] = []
                aikt[slice_hash].append(yik_idx)
    return (yik, aikt)

#### Test

In [15]:
requests = [
    {"free_windows_dict": {"t1": [{"start": 0, "end": 23}, {"start": 27, "end": 88}]},
     "duration": 10,
     "resID": 3,
     "priority": 5},
    {"free_windows_dict": {"t1": [{"start": 24, "end": 63}]},
     "duration": 13,
     "resID": 1,
     "priority": 4}
]

build_data_structures(requests, 5)

([[3, 0, 5, 't1', 0],
  [3, 1, 5, 't1', 0],
  [3, 2, 5, 't1', 0],
  [3, 3, 5, 't1', 0],
  [3, 4, 5, 't1', 0],
  [3, 5, 5, 't1', 0],
  [3, 6, 5, 't1', 0],
  [3, 7, 5, 't1', 0],
  [3, 8, 5, 't1', 0],
  [3, 9, 5, 't1', 0],
  [3, 10, 5, 't1', 0],
  [3, 11, 5, 't1', 0],
  [3, 12, 5, 't1', 0],
  [3, 13, 5, 't1', 0],
  [1, 0, 4, 't1', 0],
  [1, 1, 4, 't1', 0],
  [1, 2, 4, 't1', 0],
  [1, 3, 4, 't1', 0],
  [1, 4, 4, 't1', 0],
  [1, 5, 4, 't1', 0],
  [1, 6, 4, 't1', 0]],
 {'resource_t1_start_0_length_5': [0],
  'resource_t1_start_5_length_5': [0, 1],
  'resource_t1_start_10_length_5': [1, 2],
  'resource_t1_start_15_length_5': [2],
  'resource_t1_start_25_length_5': [3, 14, 15],
  'resource_t1_start_30_length_5': [3, 4, 14, 15, 16],
  'resource_t1_start_35_length_5': [3, 4, 5, 14, 15, 16, 17],
  'resource_t1_start_40_length_5': [5, 6, 16, 17, 18],
  'resource_t1_start_45_length_5': [6, 7, 17, 18, 19],
  'resource_t1_start_50_length_5': [7, 8, 18, 19, 20],
  'resource_t1_start_55_length_5': [8, 

### Legacy Code

In [23]:
        # first we need to build up the list of discretized slices that each
        # reservation can begin in. These are represented as attributes
        # that get attached to the reservation object. 
        # The new attributes are: 
        # slices_dict
        # internal_starts_dict
        # and the dicts are keyed by resource. 
        # the description of slices and internal starts is in intervals.py

        print(self.Yik)
        print(self.aikt)
        sys.exit()

        for r in self.reservation_list:
            r.Yik_entries = []
            r.possible_starts = []
            print("free_windows_dict")
            print(r.free_windows_dict)
            for resource in r.free_windows_dict.keys():
                r.possible_starts.extend(self.get_slices( r.free_windows_dict[resource], resource, r.duration))
            # reorder PossibleStarts
            r.possible_starts.sort()
            # build aikt
            w_idx = 0
            for ps in r.possible_starts:
                Yik_idx = len(self.Yik)
                r.Yik_entries.append(Yik_idx)
                # set the initial warm start solution
                scheduled = 0
                if r.previous_solution_reservation and r.previous_solution_reservation.scheduled_start == ps.internal_start and r.previous_solution_reservation.scheduled_resource == ps.resource:
                    scheduled = 1
                # now w_idx is the index into r.possible_starts, which have
                # been reordered by time.
                self.Yik.append([r.resID, w_idx, r.priority, ps.resource, scheduled])
                w_idx += 1
                # build aikt
                for s in ps.all_slice_starts:
                    key, exists = self.hash_slice(s, ps.resource, self.time_slicing_dict[ps.resource][1])
    #                        if key in self.aikt:
                    if exists:
                        self.aikt[key].append(Yik_idx)
                    else:
                        self.aikt[key] = [Yik_idx]

NameError: name 'self' is not defined

#### Old Code

In [20]:
# Translate Requests into widx slices
slices = {}

for request_id in range(len(requests)):
    r = requests[request_id]
    window_start = r["window_start"]
    window_end = r["window_end"]
    length = r["length"]
    priority = r["priority"]

    # print(request_id, window_start, window_end, length)

    for r, r_slices in resource_slices.items():
        window_slices = list(range(window_start, window_end+1, slice_size))
        resource_window_slices = [t for t in r_slices if t in window_slices] # Will need to redo this part

        # print(window_slices)
        # print(resource_window_slices)
        # print(r_slices)

        possible_windows = []
        for t1 in resource_window_slices:
            valid = True
            for section in range(math.ceil(length / slice_size)):
                if (t1 + section) not in resource_window_slices:
                    valid = False
                    break

            if valid:
                print("valid")
                possible_windows.append(t1)


for rid in range(len(requests)):

    r = requests[rid]
    w_start = r["window_start"]
    w_end = r["window_end"]
    length = r["length"]
    priority = r["priority"]

    print(rid, w_start, w_end, length)
    
    for resource, hours in resource_slices.items():
        # Determine if the current start can fit in the time slices for this resource
        initial_window = range(w_start, end+1)
        print(initial_window)
        overlap = [t for t in hours if t in initial_window]
        print(overlap)
        possible_windows = []
        for t1 in overlap:
            valid = True
            for section in range(length):
                if (t1 + section) not in overlap:
                    valid = False
                    break
            
            if valid:
                # Add to possible_windows
                possible_windows.append(t1)
                
        # print(possible_windows)
        for w in possible_windows:
            name = f"TEL-{resource}_ID-{rid}_ST-{w}"
            wnum = w + section

            slices[name] = {
                "rid": rid,
                "priority": priority,
                "resource": resource,
                "start": w,
                "duration": length
            }

    print()

print("HELLO")

for s in slices:
    print(s)
    # print(slices[s])
    print()
# print(aikt) 

0 31190 77678 8946
range(31190, 82966)
[31200, 31500, 31800, 32100, 32400, 32700, 33000, 33300, 33600, 33900, 34200, 34500, 34800, 35100, 35400, 35700, 36000, 36300, 36600, 36900, 37200, 37500, 37800, 38100, 38400, 38700, 39000, 39300, 39600, 39900, 40200, 40500, 40800, 41100, 41400, 41700, 42000, 42300, 42600, 42900, 43200, 43500, 43800, 44100, 44400, 44700, 45000, 45300, 45600, 45900, 46200, 46500, 46800, 47100, 47400, 47700, 48000, 48300, 48600, 48900, 49200, 49500, 49800, 50100, 50400, 50700, 51000, 51300, 51600, 51900, 52200, 52500, 52800, 53100, 53400, 53700, 54000, 54300, 54600, 54900, 55200, 55500, 55800, 56100, 56400, 56700, 57000, 57300, 57600, 57900, 58200, 58500, 58800, 59100, 59400, 59700, 60000, 60300, 60600, 60900, 61200, 61500, 61800, 62100, 62400, 62700, 63000, 63300, 63600, 63900, 64200, 64500, 64800, 65100, 65400, 65700, 66000, 66300, 66600, 66900, 67200, 67500, 67800, 68100, 68400, 68700, 69000, 69300, 69600, 69900, 70200, 70500, 70800, 71100, 71400, 71700, 72000, 7

In [21]:
aikt = {}

for slice_num, slice_id in enumerate(slices):
    slice = slices[slice_id]
    for segment in range(slice["duration"]):
        win_id = slice["start"] + segment
        if win_id in aikt:
            aikt[win_id].append(slice_num)
        else:
            aikt[win_id] = [slice_num]

# for k in sorted(aikt.keys()):
    # print(k, aikt[k])

### Build Model

In [16]:
def build_model(yik, aikt):
    m = Model("Test Schedule")

    requestLocations = tuplelist()
    scheduled_vars = []

    for r in yik:
        var = m.addVar(vtype=GRB.BINARY, name=str(r[0]))
        scheduled_vars.append(var)
        request_locations.append((r[0], r[1], r[2], r[3], var)) # resID, start_w_idx, priority, resource, isScheduled?
    m.update()

    # Constraint 4: Each request only scheduled once
    for rid in requests:
        match = requestLocations.select(rid, '*', '*', '*', '*', '*')
        nscheduled = quicksum([isScheduled for reqid, wnum, priority, resource, isScheduled in match])
        m.addConstr(nscheduled <= 1, f'one_per_reqid_constraint_{rid}')
    m.update()

    # Constraint 3: Each timeslice should only have one request in it
    for s in aikt.keys():
        match = tuplelist()
        for timeslice in aikt[s]:
            match.append(request_locations[timeslice])
        nscheduled = quicksum(isScheduled for reqid, winidx, priority, resource, isScheduled in match)
        m.addConstr(nscheduled <= 1, f"one_per_slice_constrain_{s}")

    objective = quicksum([isScheduled * (priority + 0.1/(wnum+1.0)) for req, start, priority, resource, isScheduled in requestLocations])

    m.setObjective(objective)
    m.modelSense(GRB.MAXIMIZE)

    # Implement a timelimit? (Do I actually want to do this?)
    if timelimit > 0:
        m.params.timeLimit = timelimit

    # Set the tolerance of the solution
    m.params.MIPGap = 0.01

    # Set the Method of solving the root relaxation of the MIPs model to concurrent
    m.params.Method = 3

    m.update()

    m.write("test_model.mps")

#### Load Model

In [19]:
def load_model(model_name="test_model.mps"):
    model = gurobi_read_model(model_name)
    return model

## Input Parameters

### Set request-generation parameters

In [4]:
num_requests = 20
request_min_length = 30
request_max_length = 60*60*3
priority_min = 1
priority_max = 10

### Set up resources and availability windows TODO

In [18]:
resources = ["t1"] #,"t2", "t3"
timeslices = [i for i in range(START_TIME, START_TIME + horizon, slice_size)]
print(len(timeslices))
# timeslices = [i for i in range(300)]
print(resources)
print(timeslices)

288
['t1']
[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000, 3300, 3600, 3900, 4200, 4500, 4800, 5100, 5400, 5700, 6000, 6300, 6600, 6900, 7200, 7500, 7800, 8100, 8400, 8700, 9000, 9300, 9600, 9900, 10200, 10500, 10800, 11100, 11400, 11700, 12000, 12300, 12600, 12900, 13200, 13500, 13800, 14100, 14400, 14700, 15000, 15300, 15600, 15900, 16200, 16500, 16800, 17100, 17400, 17700, 18000, 18300, 18600, 18900, 19200, 19500, 19800, 20100, 20400, 20700, 21000, 21300, 21600, 21900, 22200, 22500, 22800, 23100, 23400, 23700, 24000, 24300, 24600, 24900, 25200, 25500, 25800, 26100, 26400, 26700, 27000, 27300, 27600, 27900, 28200, 28500, 28800, 29100, 29400, 29700, 30000, 30300, 30600, 30900, 31200, 31500, 31800, 32100, 32400, 32700, 33000, 33300, 33600, 33900, 34200, 34500, 34800, 35100, 35400, 35700, 36000, 36300, 36600, 36900, 37200, 37500, 37800, 38100, 38400, 38700, 39000, 39300, 39600, 39900, 40200, 40500, 40800, 41100, 41400, 41700, 42000, 42300, 42600, 42900, 43200, 43500, 43800,

### Generate Observation Requests (Randomly)

In [7]:
# Generate random requests
requests = {}

for i in range(num_requests):
    # NOTE: Need to come up with a method of cadencing these, as an observing window cannot exist continuously for 3 days...
    # Also it could be for a couple of hours per night, as opposed to just one long stretch.
    
    t1 = random.randint(START_TIME, START_TIME + horizon)
    t2 = random.randint(START_TIME, START_TIME + horizon)
    start = min([t1, t2])
    end = max([t1, t2])
    
    length = min(random.randint(request_min_length, request_max_length), end-start)
    
    priority = random.randint(priority_min, priority_max)
    
    requests[i] = {
            "window_start": start,
            "window_end": end,
            "length": length,
            "priority": priority
        }

for r in requests.items():
    print(r)

(0, {'window_start': 31190, 'window_end': 77678, 'length': 8946, 'priority': 3})
(1, {'window_start': 48490, 'window_end': 79157, 'length': 7796, 'priority': 10})
(2, {'window_start': 8588, 'window_end': 79377, 'length': 245, 'priority': 8})
(3, {'window_start': 33994, 'window_end': 72192, 'length': 3869, 'priority': 4})
(4, {'window_start': 61638, 'window_end': 70906, 'length': 9035, 'priority': 8})
(5, {'window_start': 52053, 'window_end': 83763, 'length': 2497, 'priority': 4})
(6, {'window_start': 19873, 'window_end': 83212, 'length': 8601, 'priority': 7})
(7, {'window_start': 1985, 'window_end': 8392, 'length': 2641, 'priority': 10})
(8, {'window_start': 5608, 'window_end': 39487, 'length': 538, 'priority': 5})
(9, {'window_start': 61964, 'window_end': 77955, 'length': 6380, 'priority': 7})
(10, {'window_start': 51768, 'window_end': 75616, 'length': 7314, 'priority': 3})
(11, {'window_start': 12773, 'window_end': 47909, 'length': 617, 'priority': 3})
(12, {'window_start': 28440, 'w

## Build Model (OUTDATED)

In [ ]:
m = Model("Test Schedule")

In [ ]:
requestLocations = tuplelist()
scheduled_vars = []

for r in yik:
    

for r_name, r in slices.items():
    var = m.addVar(vtype=GRB.BINARY, name=str(r_name))
    scheduled_vars.append(var)
    requestLocations.append((r["rid"], r["start"], r["priority"], r["resource"], var))

m.update()

In [ ]:
# I DON'T KNOW WHY WE NEED THIS ONE
# for i, r in enumerate(slices):
    # print(r)
    # scheduled_vars[i].start = r["start"]
# m.update()

In [ ]:
# Constraint 1: One-of (only schedule one of these)
# Are we going to include these constraints in our final model? If there's no special cases, then they just turn into 
# Constraint 4 (which is why there is a skip_constraint2 flag).

In [ ]:
# # Constraint 2: And (only schedule if they can ALL be scheduled)
# # I'm going to use this to ensure that all of an observation is schedule

# NOTE: Not what this is used for. Each Request/Resource/Start has a unique IsScheduled variable, not each timeslice.

# Use this to schedule cadenced observations? And what does that mean if one of an AND condition has already been observed?
# Are the rest of the observations now fixed in place?
# Maybe this is too complicated to inject in the first run, especially as I don't  know how LCO implements these things - I don't know what observation
# requests get tagged with the AND or OR constraints (1 and 2), and how they are handled internally once part of the AND condition is fulfilled.

In [8]:
# Constraint 4: Each request only scheduled once
for rid in requests:
    match = requestLocations.select(rid, '*', '*', '*', '*', '*')
    nscheduled = quicksum([isScheduled for reqid, wnum, priority, resource, isScheduled in match])
    m.addConstr(nscheduled <= 1, f'one_per_reqid_constraint_{rid}')
m.update()

NameError: name 'requestLocations' is not defined

In [9]:
# Constraint 3: Each timeslice should only have one request in it
for r in resource_slices:
    for wnum in resource_slices[r]:
        match = requestLocations.select('*', wnum, '*', r, '*', '*')
        if len(match) == 0:
            continue
        nscheduled = quicksum(isScheduled for i, w, p, r, isScheduled in match)
        m.addConstr(nscheduled <= 1, f'one_per_slice_constraint_{r}_{wnum}')
m.update()

TypeError: 'int' object is not iterable

In [30]:
for s in aikt.keys():
    match = tuplelist()
    for slice in aikt[s]:
        match.append(requestLocations[slice])
        print(match)
    nscheduled = quicksum(isScheduled for rid, winidx, priority, resource, isScheduled in match)
    m.addConstr(nscheduled <= 1, 'one_per_slice_constraint_' + s)

<gurobi.tuplelist (1 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
>
<gurobi.tuplelist (2 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
>
<gurobi.tuplelist (3 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
>
<gurobi.tuplelist (4 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
 ( 4 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-4_ST-5> )
>
<gurobi.tuplelist (5 tuples, 5 values each):
 ( 0 , 5 , 5 , t1 , <gurobi.Var TEL-t1_ID-0_ST-5> )
 ( 1 , 5 , 4 , t1 , <gurobi.Var TEL-t1_ID-1_ST-5> )
 ( 2 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-2_ST-5> )
 ( 4 , 5 , 2 , t1 , <gurobi.Var TEL-t1_ID-4_ST-5> )
 ( 6 , 5 , 1 , t1 , <gurobi.Var TEL-t1_

In [60]:
objective = quicksum([isScheduled * (priority + 0.1/(wnum+1.0)) for req, start, priority, resource, isScheduled in requestLocations])

In [61]:
m.setObjective(objective)
m.modelSense = GRB.MAXIMIZE
m.params.MIPGap = 0.01
m.params.Method = 3
m.update()

Set parameter MIPGap to value 0.01
Set parameter Method to value 3


### Write Model to file

In [22]:
m.write("test_model.mps")

NameError: name 'm' is not defined

### Solve Model

In [ ]:
m.optimize()

## Model

### Build Model

In [20]:
build_model(yik, aikt)

NameError: name 'yik' is not defined

### Solve Model - Gurobi

In [22]:
model = load_model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Read MPS format model from file test_model.mps
Reading time = 0.01 seconds
Test: 20 rows, 0 columns, 0 nonzeros


## Interpret Results

In [23]:
rxf = [isScheduled.x for rid, start, priority, resource, isScheduled in requestLocations]

NameError: name 'requestLocations' is not defined

In [65]:
print("ID, w, p, reso, s")
for i in range(len(requestLocations)):
    if rxf[i] == 1:
        print(requestLocations[i])

ID, w, p, reso, s
(1, 207, 4, 't1', <gurobi.Var TEL-t1_ID-1_ST-207 (value 1.0)>)
(5, 198, 5, 't1', <gurobi.Var TEL-t1_ID-5_ST-198 (value 1.0)>)
(7, 37, 4, 't3', <gurobi.Var TEL-t3_ID-7_ST-37 (value 1.0)>)
(8, 96, 2, 't1', <gurobi.Var TEL-t1_ID-8_ST-96 (value 1.0)>)
(10, 127, 2, 't3', <gurobi.Var TEL-t3_ID-10_ST-127 (value 1.0)>)
(11, 239, 3, 't1', <gurobi.Var TEL-t1_ID-11_ST-239 (value 1.0)>)
(19, 249, 3, 't3', <gurobi.Var TEL-t3_ID-19_ST-249 (value 1.0)>)
(20, 20, 4, 't3', <gurobi.Var TEL-t3_ID-20_ST-20 (value 1.0)>)
(21, 80, 2, 't1', <gurobi.Var TEL-t1_ID-21_ST-80 (value 1.0)>)
(22, 193, 4, 't3', <gurobi.Var TEL-t3_ID-22_ST-193 (value 1.0)>)
(25, 155, 5, 't3', <gurobi.Var TEL-t3_ID-25_ST-155 (value 1.0)>)
(26, 199, 5, 't2', <gurobi.Var TEL-t2_ID-26_ST-199 (value 1.0)>)
(27, 179, 4, 't3', <gurobi.Var TEL-t3_ID-27_ST-179 (value 1.0)>)
(28, 192, 4, 't2', <gurobi.Var TEL-t2_ID-28_ST-192 (value 1.0)>)
(29, 46, 4, 't3', <gurobi.Var TEL-t3_ID-29_ST-46 (value 1.0)>)
(30, 165, 1, 't3', <gurob

In [67]:
requests

{0: {'window_start': 94, 'window_end': 197, 'length': 47, 'priority': 3},
 1: {'window_start': 207, 'window_end': 233, 'length': 26, 'priority': 4},
 2: {'window_start': 51, 'window_end': 110, 'length': 41, 'priority': 3},
 3: {'window_start': 3, 'window_end': 186, 'length': 29, 'priority': 1},
 4: {'window_start': 234, 'window_end': 271, 'length': 37, 'priority': 5},
 5: {'window_start': 128, 'window_end': 253, 'length': 35, 'priority': 5},
 6: {'window_start': 98, 'window_end': 251, 'length': 20, 'priority': 1},
 7: {'window_start': 37, 'window_end': 248, 'length': 29, 'priority': 4},
 8: {'window_start': 84, 'window_end': 103, 'length': 17, 'priority': 2},
 9: {'window_start': 287, 'window_end': 295, 'length': 8, 'priority': 1},
 10: {'window_start': 91, 'window_end': 102, 'length': 11, 'priority': 2},
 11: {'window_start': 232, 'window_end': 289, 'length': 8, 'priority': 3},
 12: {'window_start': 98, 'window_end': 188, 'length': 34, 'priority': 1},
 13: {'window_start': 0, 'window_